In [17]:
from functools import reduce
import pandas as pd

The Dataverse team is planning to add  additions to the dataset deposit form in the Harvard Dataverse Repository, we need to know

In Dataverse repositories, how many datasets have funding metadata, and among these, how many have funding names? Which users include funding metadata in their deposits most often?

## Prepare the data

In [2]:
# Import CSV file that lists PIDs of all datasets and which Dataaverse installation
# they're published in
datasetPIDsDF = pd.read_csv(
    'dataset_pids_from_most_known_dataverse_installations.csv',
    sep=',', na_filter=False)

# Remove the PIDs of datasets whose metadata could not be saved, i.e. dataverse_json_export_saved is FALSE
datasetPIDsDF = (datasetPIDsDF
    .query('(dataverse_json_export_saved == True)')
    .reset_index(drop=True, inplace=False)
    )

datasetPIDsDF = datasetPIDsDF.drop(columns=[
    'dataset_pid', 'dataverse_name', 'dataverse_json_export_saved'])

datasetPIDsDF.head()

,installation,dataset_pid_url,dataverse_alias
0,CIDACS,https://doi.org/10.57833/cidacs/WV4JWB,ProjZika
1,CIDACS,https://doi.org/10.57833/cidacs/TGAIVO,ProjZika
2,CIDACS,https://doi.org/10.57833/cidacs/0FZP7V,ProjZika
3,CIDACS,https://doi.org/10.57833/cidacs/5BMSIX,ProjGates0
4,CIDACS,https://doi.org/10.57833/cidacs/I56ZVA,ProjGates0


In [3]:
# Import basic metadata of all dataset versions
datasetVersionMetadataDF = pd.read_csv(
    'basic_metadata_2022.10.02-2022.10.03.csv',
    sep=',', na_filter=False)

# Remove the dataset_publication_date, dataset_version_state and publisher columns
datasetVersionMetadataDF = datasetVersionMetadataDF.drop(columns=[
    'dataset_pid', 'dataset_publication_date', 'dataset_version_state', 'publisher'])

# Make sure values in date columns are interpreted as dates
dateColumns = ['dataset_version_create_time']
datasetVersionMetadataDF[dateColumns] = datasetVersionMetadataDF[dateColumns].apply(pd.to_datetime)

datasetVersionMetadataDF.head()

,dataset_pid_url,dataset_version_number,dataset_version_create_time
0,https://doi.org/10.48370/OFD/DBJUEM,1.0,2022-05-25 10:17:50+00:00
1,https://doi.org/10.21410/7E4/4WG94W,2.1,2020-05-13 16:06:28+00:00
2,https://doi.org/10.7910/DVN/5PRYPC,4.0,2020-06-17 23:49:50+00:00
3,https://doi.org/10.17026/dans-2zm-dsmz,1.0,2022-01-27 20:15:47+00:00
4,https://doi.org/10.7910/DVN/VIE1H,248.0,2015-06-18 19:49:12+00:00


In [4]:
print(f'Count of datasets: {len(datasetPIDsDF)}')
print(f'Count of dataset versions: {len(datasetVersionMetadataDF)}')


Count of datasets: 340857
Count of dataset versions: 473910


In [5]:
# From the datasetVersionMetadataDF, lets use the version publication dates to get the PIDs and database version numbers of only the most recently published versions of each dataset. The resulting dataframe should contain the same number of rows as the datasetPIDsDF.

latestDatasetVersionsDF = (datasetVersionMetadataDF
                 .iloc[datasetVersionMetadataDF
                 .groupby('dataset_pid_url')['dataset_version_create_time']
                 .agg(pd.Series.idxmax)]
                 .reset_index(drop=True, inplace=False))

In [6]:
latestDatasetVersionsDF.head()

,dataset_pid_url,dataset_version_number,dataset_version_create_time
0,http://dx.doi.org/10.26193/00HBWG,2.0,2019-05-13 08:43:01+00:00
1,http://dx.doi.org/10.26193/01P0AI,2.0,2019-05-13 06:23:26+00:00
2,http://dx.doi.org/10.26193/04F7C1,2.0,2019-05-13 10:02:15+00:00
3,http://dx.doi.org/10.26193/07R31R,2.0,2019-05-13 06:50:29+00:00
4,http://dx.doi.org/10.26193/0AF6TZ,5.0,2022-02-02 23:01:26+00:00


In [7]:
print(f'Count of datasets: {len(datasetPIDsDF)}')
print(f'Count of rows in latestDatasetVersionsDF: {len(latestDatasetVersionsDF)}')

Count of datasets: 340857
Count of rows in latestDatasetVersionsDF: 340857


In [36]:
# Join the latestDatasetVersionsDF and the datasetPIDsDF to add the installation column,
# so we know which installations published each dataset

# Merge the two dataframes
basicDatasetMetadataDF = (pd
    .merge(latestDatasetVersionsDF, datasetPIDsDF,
        how='inner',
        on=['dataset_pid_url'])
    .reset_index(drop=True, inplace=False))
print(len(basicDatasetMetadataDF))

340857


In [37]:
basicDatasetMetadataDF.head()

,dataset_pid_url,dataset_version_number,dataset_version_create_time,installation,dataverse_alias
0,http://dx.doi.org/10.26193/00HBWG,2.0,2019-05-13 08:43:01+00:00,ADA Dataverse,NA
1,http://dx.doi.org/10.26193/01P0AI,2.0,2019-05-13 06:23:26+00:00,ADA Dataverse,NA
2,http://dx.doi.org/10.26193/04F7C1,2.0,2019-05-13 10:02:15+00:00,ADA Dataverse,NA
3,http://dx.doi.org/10.26193/07R31R,2.0,2019-05-13 06:50:29+00:00,ADA Dataverse,NA
4,http://dx.doi.org/10.26193/0AF6TZ,5.0,2022-02-02 23:01:26+00:00,ADA Dataverse,NA


In [10]:
# basicDatasetMetadataDF.to_csv('basicDatasetMetadataDF.csv', index=False)

In [22]:
# Import funding metadata
grantInformationMetadataDF = pd.read_csv(
    'grant_information(citation)_2022.10.02-2022.10.03.csv',
    sep=',', na_filter=False)

# Remove the dataset_pid column
grantInformationMetadataDF = grantInformationMetadataDF.drop(columns=['dataset_pid'])
print(len(grantInformationMetadataDF))

124072


In [40]:
# Join grantInformationMetadataDF with basicDatasetMetadataDF to retain metadata of
# each dataset's latest version

grantInformationLatestVersionDF = (pd
    .merge(grantInformationMetadataDF, basicDatasetMetadataDF,
        how='inner',
        on=['dataset_pid_url', 'dataset_version_number'])
    .reset_index(drop=True, inplace=False))

grantInformationLatestVersionDF = grantInformationLatestVersionDF.drop(columns=[
    'dataset_version_create_time', 'installation', 'dataverse_alias'])

grantInformationLatestVersionDF.head()

,dataset_pid_url,dataset_version_number,grantNumberAgency,grantNumberValue
0,https://doi.org/10.17026/dans-2zm-dsmz,1.0,NWO,380-60-007
1,https://doi.org/10.17026/dans-xq9-4s24,1.0,NWO,380-60-007
2,https://doi.org/10.17026/dans-z2f-qdzn,1.0,NWO,380-60-007
3,https://doi.org/10.17026/dans-xg6-z9ke,1.0,NWO,380-60-007
4,https://doi.org/10.7910/DVN/IJEJGR,1.0,Bill and Melinda Gates Foundation (BMGF),


In [24]:
# Import contributor metadata, where a funding agency might be listed as a contributor
contributorMetadataDF = pd.read_csv(
    'contributor(citation)_2022.10.02-2022.10.03.csv',
    sep=',', na_filter=False)

# Remove the dataset_pid column
contributorMetadataDF = contributorMetadataDF.drop(columns=['dataset_pid'])

print(len(contributorMetadataDF))

126012


In [41]:
# Join contributorMetadataDF with basicDatasetMetadataDF to retain metadata of
# each dataset's latest version

contributorLatestVersionDF = (pd
    .merge(contributorMetadataDF, basicDatasetMetadataDF,
        how='inner',
        on=['dataset_pid_url', 'dataset_version_number'])
    .reset_index(drop=True, inplace=False))

contributorLatestVersionDF = contributorLatestVersionDF.drop(columns=[
    'dataset_version_create_time', 'installation', 'dataverse_alias'])

contributorLatestVersionDF.head()

,dataset_pid_url,dataset_version_number,contributorType,contributorName
0,https://doi.org/10.7910/DVN/IJEJGR,1.0,Data Collector,Institute of Social and Medicine Studies (ISMS)
1,https://doi.org/10.7910/DVN/IJEJGR,1.0,Funder,Bill and Melinda Gates Foundation (BMGF)
2,https://doi.org/10.7910/DVN/IJEJGR,1.0,Researcher,"Kim, Sunny (International Food Policy Research Institute (IFPRI))"
3,https://doi.org/10.7910/DVN/IJEJGR,1.0,Researcher,"Menon, Purnima (International Food Policy Research Institute (IFPRI))"
4,https://doi.org/10.7910/DVN/IJEJGR,1.0,Researcher,"Nguyen, Phuong Hong (International Food Policy Research Institute (IFPRI))"


In [42]:
# Combine the basicDatasetMetadataDF, grantInformationLatestVersionDF, and contributorLatestVersionDF with a
# full outer join on dataset_pid_url and dataset_version_number columns

dataframes = [basicDatasetMetadataDF, grantInformationLatestVersionDF, contributorLatestVersionDF]
indexList = ['dataset_pid_url', 'dataset_version_number']
for dataframe in dataframes:
    dataframe.set_index(indexList, inplace=True)

In [51]:
fundingDatasetMetadataInDataverseInstallationsDF = reduce(lambda left, right: left.join(right, how='outer'), dataframes)
fundingDatasetMetadataInDataverseInstallationsDF = fundingDatasetMetadataInDataverseInstallationsDF.reset_index(drop=False, inplace=False)
fundingDatasetMetadataInDataverseInstallationsDF.head()

,dataset_pid_url,dataset_version_number,dataset_version_create_time,installation,dataverse_alias,grantNumberAgency,grantNumberValue,contributorType,contributorName
0,http://dx.doi.org/10.26193/00HBWG,2.0,2019-05-13 08:43:01+00:00,ADA Dataverse,NA,NaN,NaN,Funder,"General Practice Evaluation Program, Department of Health, Housing and Community Services"
1,http://dx.doi.org/10.26193/01P0AI,2.0,2019-05-13 06:23:26+00:00,ADA Dataverse,NA,NaN,NaN,Data Collector,Roy Morgan
2,http://dx.doi.org/10.26193/04F7C1,2.0,2019-05-13 10:02:15+00:00,ADA Dataverse,NA,NaN,NaN,Funder,Australian Research Council
3,http://dx.doi.org/10.26193/04F7C1,2.0,2019-05-13 10:02:15+00:00,ADA Dataverse,NA,NaN,NaN,Data Collector,Workplace Research Centre
4,http://dx.doi.org/10.26193/07R31R,2.0,2019-05-13 06:50:29+00:00,ADA Dataverse,NA,NaN,NaN,NaN,NaN


In [52]:
print('Number of datasets in fundingDatasetMetadataInDataverseInstallationsDF: %s' % (len(pd.unique(fundingDatasetMetadataInDataverseInstallationsDF['dataset_pid_url']))))

Number of datasets in fundingDatasetMetadataInDataverseInstallationsDF: 340857


## Exploring the data

Now that we've got the funding metadata of the latest versions of all datasets in the Dataverse installations, let's start answering our questions:
- In the Harvard Dataverse Repository, which collections have datasets with the most funding metadata? This will help us figure out who to learn from when we make changes to how funding metadata is entered.
- Which funder agency names are entered most often? Knowing that might help us figure out how effective our efforts to standardize funder agency name metadata could be? For example, do the changes to the metadata fields (the "CV javascript") make it easier for depositors to enter the most popular funder agency names? How much easier?

In [61]:
# Lets start by creating a dataframe containing only metadata of datasets published in the Harvard Dataverse Repository
datasetInHDVDF = (fundingDatasetMetadataInDataverseInstallationsDF
    .query('(installation == "Harvard Dataverse")')
    .drop(columns=['installation'])
    .reset_index(drop=True, inplace=False)
    )

datasetInHDVDF.head()

,dataset_pid_url,dataset_version_number,dataset_version_create_time,dataverse_alias,grantNumberAgency,grantNumberValue,contributorType,contributorName
0,https://doi.org/10.7910/DVN/00234,1.0,2014-03-23 22:12:34+00:00,restat,NaN,NaN,NaN,NaN
1,https://doi.org/10.7910/DVN/004HG6,1.0,2020-08-28 15:42:11+00:00,levy_photos_2012,The Leon Levy Foundation,,NaN,NaN
2,https://doi.org/10.7910/DVN/005SCF,1.0,2021-08-26 03:17:40+00:00,levy_photos_2015,The Leon Levy Foundation,,NaN,NaN
3,https://doi.org/10.7910/DVN/006UPU,1.0,2021-08-13 18:15:36+00:00,levy_photos_2014,The Leon Levy Foundation,,NaN,NaN
4,https://doi.org/10.7910/DVN/007GT,5.0,2017-02-05 23:43:33+00:00,antislaverypetitionsma,NaN,NaN,NaN,NaN


In [62]:
print('Number of datasets in datasetInHDVDF: %s' % (len(pd.unique(datasetInHDVDF['dataset_pid_url']))))

Number of datasets in datasetInHDVDF: 80278


In [63]:
# Now let's keep the metadata of the datasets that have funding metadata in the three fields: grantNumberAgency, grantNumberValue, and contributorName when contributorType is "Funder"
fundingDatasetMetadataInHDVDF = (
    datasetInHDVDF.query(
        'grantNumberAgency == grantNumberAgency\
        or grantNumberValue == grantNumberValue\
        or (contributorType == "Funder" and contributorName == contributorName)')
     .reset_index(drop=True, inplace=False)
     )


In [58]:
fundingDatasetMetadataInHDVDF.head()

,dataset_pid_url,dataset_version_number,dataset_version_create_time,dataverse_alias,grantNumberAgency,grantNumberValue,contributorType,contributorName
0,https://doi.org/10.7910/DVN/004HG6,1.0,2020-08-28 15:42:11+00:00,levy_photos_2012,The Leon Levy Foundation,,NaN,NaN
1,https://doi.org/10.7910/DVN/005SCF,1.0,2021-08-26 03:17:40+00:00,levy_photos_2015,The Leon Levy Foundation,,NaN,NaN
2,https://doi.org/10.7910/DVN/006UPU,1.0,2021-08-13 18:15:36+00:00,levy_photos_2014,The Leon Levy Foundation,,NaN,NaN
3,https://doi.org/10.7910/DVN/00IIWB,1.0,2022-03-16 01:03:47+00:00,levy_photos_2013,The Leon Levy Foundation,,NaN,NaN
4,https://doi.org/10.7910/DVN/00VQS7,1.0,2021-12-16 19:14:00+00:00,levy_photos_1986,The Leon Levy Foundation,,NaN,NaN


In [59]:
print('Number of datasets in fundingDatasetMetadataInHDVDF: %s' % (len(pd.unique(fundingDatasetMetadataInHDVDF['dataset_pid_url']))))

Number of datasets in fundingDatasetMetadataInHDVDF: 30228


So of the 80,278 datasets in the Harvard Dataverse Repository as of early October 2022, 30,228 had metadata about funding in one of the three fields where we expect it.

Which collections have the most datasets with metadata?

In [75]:
datasetsInHDVByCollection = (
    fundingDatasetMetadataInHDVDF[['dataset_pid_url', 'dataverse_alias']]
    .drop_duplicates()
    .groupby(['dataverse_alias']).count()
    .rename(columns={'dataset_pid_url': 'count_of_datasets'})
    .sort_values(by=['count_of_datasets'], ascending=False)
    .reset_index(drop=False, inplace=False)
)

datasetsInHDVByCollection.head(10)

,dataverse_alias,count_of_datasets
0,levy_photos_2012,3203
1,levy_photos_2015,3147
2,levy_photos_2016,2569
3,levy_photos_2013,2187
4,levy_photos_2011,1942
5,levy_photos_2004,1242
6,levy_photos_1998,1233
7,levy_photos_1997,1173
8,levy_photos_2000,1032
9,levy_photos_2014,954


The datasets in the collections with "leon_photos..." aliases were created using the Dataverse APIs, and the funder metadata is "The Leon Levy Foundation" in the Funding Information Name field. The Funding Information Number fields are blank.

For this data, is there a number for the funding from The Leon Levy Foundation? Something to ask the depositors

Let's see which collections have the most datasets when we remove these "leon_photos..." collections

In [76]:
datasetsInHDVByCollection_NoLevy = (
    datasetsInHDVByCollection.query(
        '~dataverse_alias.str.contains("levy_photos").values')
        .reset_index(drop=True, inplace=False)
    )

datasetsInHDVByCollection_NoLevy.head(10)

,dataverse_alias,count_of_datasets
0,IFPRI,339
1,harvard,201
2,AfricaRISING,136
3,worldfish,113
4,CIAT,108
5,ipa,65
6,AllianceBioversityCIAT,50
7,IMASC_Publication_Data,42
8,cenaptnmr,40
9,LakeVicFish,30


The IFPRI, AfricaRISING, and worldfish collections have the most datasets with funding metadata. ("harvard" is the repository's main collection, where anyone can add data.) What have the depositors of those datasets entered in the metadata?


In [77]:
# fundingDatasetMetadataInHDVDF.to_csv('fundingDatasetMetadataInHDVDF.csv', index=False)

In [ ]:
fundingDatasetMetadataInIFPRI = (fundingDatasetMetadataInHDVDF
    .query(
        'dataverse_alias == "IFPRI"\
        or dataverse_alias == "AfricaRISING"')

)

In [84]:
datasetCountByFundingAgencyNamesIFPRI = (
    fundingDatasetMetadataInIFPRI[['dataset_pid_url', 'grantNumberAgency']]
    .query(
        'grantNumberAgency == grantNumberAgency')
    .drop_duplicates()
    .groupby(['grantNumberAgency']).count()
    .rename(columns={'dataset_pid_url': 'count_of_datasets'})
    .sort_values(by=['count_of_datasets'], ascending=False)
    .reset_index(drop=False, inplace=False)
)


In [86]:
datasetCountByFundingAgencyNamesIFPRI.head(10)

,grantNumberAgency,count_of_datasets
0,United States Agency for International Development (USAID),226
1,Bill and Melinda Gates Foundation (BMGF),64
2,World Bank,24
3,Bill and Melinda Gates Foundation,24
4,Bill & Melinda Gates Foundation (BMGF),23
5,United Nations International Children's Emergency Fund (UNICEF),17
6,Tanoto Foundation,17
7,Irish Aid,17
8,"CGIAR Research Program on Policies, Institutions, and Markets (PIM)",16
9,Global Affairs Canada (GAC),12


In [ ]:

funderNamesList = list(set(fundingDatasetMetadataInIFPRI["grantNumberAgency"].values.tolist()))
print(funderNamesList)